In [1]:
import sys
sys.path.insert(0, '..')

import torch.nn as nn
from torch.utils.data import DataLoader
from utils.eye_dataset import *
from eye_classifier import *
import torchvision.transforms as transforms

### The Training Dataset

We use the utility class EyeImageDataset() to load the training dataset based on the metadata CSV file and
the target image folder

The training images will be loaded as needed due to memory constraints.

First, we resize our images to 512x512, so we apply some transforms on our dataset.

Next, we normalize the dataset and then it is ready to be trained / tested

In [2]:
base_dir = "../../data"
image_dir_training = f"{base_dir}/ODIR-5K/training"
image_dir_testing = f"{base_dir}/ODIR-5K/testing"
csv_file = f'{base_dir}/ODIR-5K/data.csv'

input_size = 512

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_training, data_info_csv_file=csv_file, transform=apply_transforms)

### Building the model

To build the model, we use the utility superclass EyeClassifier. 

We create a subclass named ResnetEyeClassifier and then we feed it with the intended model on its __init__ constructor.

Here we're building some CNN as the first layers

The last layer will not have a transfer function because we opted to use CrossEntropyLoss as the loss function. 

You can change the loss function by using set_loss_function() if you wish to


In [3]:
class CustomEyeClassifier(EyeClassifier):
    def __init__(self, num_classes: int) -> None:
        super(CustomEyeClassifier, self).__init__(model=[

            (nn.Conv2d(in_channels=3, out_channels=6,
                       kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1)),
             TransferFunction.NotApplicable),

            (nn.MaxPool2d(
                kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1)),
             TransferFunction.NotApplicable),

            (nn.Dropout(),
             TransferFunction.NotApplicable),

            (nn.Conv2d(in_channels=6, out_channels=16,
                       kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1)),
             TransferFunction.NotApplicable),

            (nn.MaxPool2d(
                kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1)),
             TransferFunction.NotApplicable),

            (nn.Dropout(),
             TransferFunction.NotApplicable),

            (nn.Linear(in_features=13456, out_features=84),
             TransferFunction.Relu),

            (nn.Linear(in_features=84, out_features=42),
             TransferFunction.Relu),

            (nn.Linear(in_features=42, out_features=num_classes),
             TransferFunction.NotApplicable),
        ])

nn = CustomEyeClassifier(num_classes=len(ds.classes))
print(nn)

CustomEyeClassifier(
  (layer 1): Conv2d(3, 6, kernel_size=(5, 5), stride=(2, 2))
  (layer 2): MaxPool2d(kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
  (layer 3): Dropout(p=0.5, inplace=False)
  (layer 4): Conv2d(6, 16, kernel_size=(5, 5), stride=(2, 2))
  (layer 5): MaxPool2d(kernel_size=(5, 5), stride=(2, 2), padding=(0, 0), dilation=(1, 1), ceil_mode=False)
  (layer 6): Dropout(p=0.5, inplace=False)
  (layer 7): Linear(in_features=13456, out_features=84, bias=True)
  (layer 8): Linear(in_features=84, out_features=42, bias=True)
  (layer 9): Linear(in_features=42, out_features=8, bias=True)
)


### Training the model

Training the model is very easy. All we have to do is to call train_model() passing the EyeImageDataset object. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [4]:
nn.train_model(ds)


training (1%) epoch 1/100, loss = 2.6364
training (2%) epoch 2/100, loss = 2.1246
training (3%) epoch 3/100, loss = 0.7268
training (4%) epoch 4/100, loss = 1.6911
training (5%) epoch 5/100, loss = 1.3497
training (6%) epoch 6/100, loss = 0.8989
training (7%) epoch 7/100, loss = 1.1256
training (8%) epoch 8/100, loss = 2.0137
training (9%) epoch 9/100, loss = 2.2820
training (10%) epoch 10/100, loss = 1.3867
training (11%) epoch 11/100, loss = 0.6704
training (12%) epoch 12/100, loss = 0.7619
training (13%) epoch 13/100, loss = 1.4886
training (14%) epoch 14/100, loss = 1.3169
training (15%) epoch 15/100, loss = 1.4462
training (16%) epoch 16/100, loss = 0.9166
training (17%) epoch 17/100, loss = 1.5332
training (18%) epoch 18/100, loss = 2.1058
training (19%) epoch 19/100, loss = 1.6965
training (20%) epoch 20/100, loss = 1.6862
training (21%) epoch 21/100, loss = 0.7722
training (22%) epoch 22/100, loss = 1.4047
training (23%) epoch 23/100, loss = 2.0299
training (24%) epoch 24/100, 

### Testing the model

Testing the model is similar to training it. All we have to do is to call test_model() passing a EyeImageDataset object pointing to the test dataset images. 

If you're having I/O constraints, you can use the set_buffer_size(n) method to cache n images in memory. Be aware that you cant use it
along with shuffle, because the cache will be constantly invalidated, unless you have a lot of memory and cache all of the images.

In [5]:
input_size = 512

apply_transforms = transforms.Compose([
    transforms.Resize(size=input_size),
    transforms.CenterCrop(size=input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

ds = EyeImageDataset(root=image_dir_testing, data_info_csv_file=csv_file, transform=apply_transforms)


In [6]:
ds.set_buffer_size(16)
nn.test_model(ds)

testing 1% [4 / 6564 files]
testing 2% [72 / 6564 files]
testing 3% [140 / 6564 files]
testing 4% [208 / 6564 files]
testing 5% [276 / 6564 files]
testing 6% [344 / 6564 files]
testing 7% [412 / 6564 files]
testing 8% [480 / 6564 files]
testing 9% [548 / 6564 files]
testing 10% [616 / 6564 files]
testing 11% [684 / 6564 files]
testing 12% [752 / 6564 files]
testing 13% [820 / 6564 files]
testing 14% [888 / 6564 files]
testing 15% [956 / 6564 files]
testing 16% [1024 / 6564 files]
testing 17% [1092 / 6564 files]
testing 18% [1160 / 6564 files]
testing 19% [1228 / 6564 files]
testing 20% [1296 / 6564 files]
testing 21% [1364 / 6564 files]
testing 22% [1432 / 6564 files]
testing 23% [1500 / 6564 files]
testing 24% [1568 / 6564 files]
testing 25% [1636 / 6564 files]
testing 26% [1704 / 6564 files]
testing 27% [1772 / 6564 files]
testing 28% [1840 / 6564 files]
testing 29% [1908 / 6564 files]
testing 30% [1976 / 6564 files]
testing 31% [2044 / 6564 files]
testing 32% [2112 / 6564 files]
tes